In [ ]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [20]:
# Importando a base de dadas tratada
df = pd.read_csv("df_atualizado.csv")

In [ ]:
# Transformando para o formato parquet
df = pd.read_parquet("df_atualizado.parquet")

In [22]:
# Informações gerais sobre as variáveis do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31883794 entries, 0 to 31883793
Data columns (total 8 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   genero                 object
 1   faixa.ano              object
 2   situacao.final         object
 3   operou.12.meses.final  object
 4   estado.civil.agrupado  object
 5   profissao.categoria    object
 6   faixa.etaria           object
 7   regiao                 object
dtypes: object(8)
memory usage: 1.9+ GB


In [23]:
#Filtrar apenas as linhas onde situacao_final é "A"
df = df[df["situacao.final"] == "A"]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27651891 entries, 2 to 31883793
Data columns (total 8 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   genero                 object
 1   faixa.ano              object
 2   situacao.final         object
 3   operou.12.meses.final  object
 4   estado.civil.agrupado  object
 5   profissao.categoria    object
 6   faixa.etaria           object
 7   regiao                 object
dtypes: object(8)
memory usage: 1.9+ GB


In [25]:
#Apagando a variável que não será utilizada no modelo final
df = df.drop(columns='situacao.final')

In [26]:
#Informações gerais dos dados que vamos usar para a regressão logística
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27651891 entries, 2 to 31883793
Data columns (total 7 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   genero                 object
 1   faixa.ano              object
 2   operou.12.meses.final  object
 3   estado.civil.agrupado  object
 4   profissao.categoria    object
 5   faixa.etaria           object
 6   regiao                 object
dtypes: object(7)
memory usage: 1.6+ GB


In [29]:
df.shape

(27651891, 7)

In [30]:
# Renomeando as variáveis
df = df.rename(columns={"operou.12.meses.final": "operou_12_meses",
                        "faixa.etaria": "faixa_etaria",
                        'faixa.ano':'faixa_ano',
                        'estado.civil.agrupado':'estado_civil',
                        'profissao.categoria':'profissao',
                        })

In [31]:
# Informações atualizadas
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27651891 entries, 2 to 31883793
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   genero           object
 1   faixa_ano        object
 2   operou_12_meses  object
 3   estado_civil     object
 4   profissao        object
 5   faixa_etaria     object
 6   regiao           object
dtypes: object(7)
memory usage: 1.6+ GB


In [17]:
variaveis = ["genero", "faixa_ano", "estado_civil", "profissao", "faixa_etaria", "regiao"]

resultado_final = pd.DataFrame()

for var in variaveis:
    tab = pd.crosstab(df[var], df["operou_12_meses"])
    
    # Pega dinamicamente os nomes das colunas (ex: "Sim" e "Não", ou 0 e 1)
    col_sim = tab.columns[1] if len(tab.columns) > 1 else tab.columns[0]
    col_nao = tab.columns[0]
    
    # calcula % dentro de cada grupo Sim, Não e Total
    tab["% Sim"] = (tab[col_sim] / tab[col_sim].sum()) * 100
    tab["% Não"] = (tab[col_nao] / tab[col_nao].sum()) * 100
    tab["% Total"] = (tab.sum(axis=1) / len(df)) * 100
    
    tab = tab[["% Sim", "% Não", "% Total"]]  # mantém apenas as % que queremos
    tab.index = [f"{var} - {idx}" for idx in tab.index]
    
    resultado_final = pd.concat([resultado_final, tab])

resultado_final = resultado_final.round(2)
resultado_final

operou_12_meses,% Sim,% Não,% Total
genero - F,30.64,23.89,24.55
genero - M,69.36,76.11,75.45
faixa_ano - 2011-2015,6.47,1.22,1.74
faixa_ano - 2016-2020,36.41,18.90,20.62
faixa_ano - 2021-2025,57.12,79.87,77.65
estado_civil - casado.uniao.estavel,3.81,2.85,2.94
estado_civil - outro,0.77,1.10,1.07
estado_civil - separado.divorciado,39.42,30.80,31.64
estado_civil - solteiro,56.00,65.25,64.35
"profissao - Administracao, Negocios e Contabilidade",19.72,24.79,24.29


In [21]:
# Estimação de um modelo logísitico binário pela função 'smf.glm'

modelo = smf.glm(formula='operou_12_meses ~ genero + faixa_ano + estado_civil + profissao+ faixa_etaria + regiao', 
                 data=df, family=sm.families.Binomial()).fit()

# Parâmetros do modelo
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              Generalized Linear Model Regression Results                               
========================================================================================================
Dep. Variable:     ['operou_12_meses[N]', 'operou_12_meses[S]']   No. Observations:             27651891
Model:                                                      GLM   Df Residuals:                 27651868
Model Family:                                          Binomial   Df Model:                           22
Link Function:                                            Logit   Scale:                          1.0000
Method:                                                    IRLS   Log-Likelihood:            -8.3539e+06
Date:                                          Wed, 24 Sep 2025   Deviance:                   1.6708e+07
Time:                                                  18:55:33   Pearson chi2:                 2.75e+07
No. Iterations:                                              11   Pseudo R-squ. (CS):            0.03541
Covariance Type:                                      nonrobust                                         
=======================================================================================================================
                                                          coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
Intercept                                               1.1029      0.006    196.303      0.000       1.092       1.114
genero[T.M]                                             0.2520      0.001    169.403      0.000       0.249       0.255
faixa_ano[T.2016-2020]                                  0.8779      0.003    266.342      0.000       0.871       0.884
faixa_ano[T.2021-2025]                                  1.7550      0.003    537.517      0.000       1.749       1.761
estado_civil[T.outro]                                   0.3278      0.008     40.561      0.000       0.312       0.344
estado_civil[T.separado.divorciado]                    -0.1007      0.004    -27.954      0.000      -0.108      -0.094
estado_civil[T.solteiro]                                0.0480      0.004     13.099      0.000       0.041       0.055
profissao[T.Agropecuaria e Meio Ambiente]               0.0332      0.009      3.685      0.000       0.016       0.051
profissao[T.Comunicacao, Artes e Midia]                -0.1818      0.005    -39.429      0.000      -0.191      -0.173
profissao[T.Direito, Seguranca e Servicos Publicos]    -0.6304      0.003   -234.771      0.000      -0.636      -0.625
profissao[T.Educacao, Pesquisa e Ciencias]             -0.4524      0.003   -172.300      0.000      -0.458      -0.447
profissao[T.Industria, Construcao e Transporte]         0.1021      0.004     28.673      0.000       0.095       0.109
profissao[T.Outros]                                    -0.2531      0.002   -136.848      0.000      -0.257      -0.250
profissao[T.Saude]                                     -0.5816      0.003   -181.739      0.000      -0.588      -0.575
profissao[T.Servicos Gerais e Atendimento]              0.1306      0.003     41.801      0.000       0.124       0.137
profissao[T.Tecnologia, Engenharia e Producao]         -0.6409      0.003   -227.202      0.000      -0.646      -0.635
faixa_etaria[T.35 a 44 anos]                           -0.2332      0.002   -129.369      0.000      -0.237      -0.230
faixa_etaria[T.AtÃ© 24 anos]                           -0.1942      0.002    -90.517      0.000      -0.198      -0.190
faixa_etaria[T.Mais de 44 anos]                        -0.1883      0.002    -95.057      0.000      -0.192      -0.184
regiao[T.nordeste]                                      0.0100      0.003      3.497      0.000       0.004       0.016
regiao[T.norte]                      

In [22]:
# Intervalos de confiança de 95%
intervalos = modelo.conf_int(alpha=0.05)
intervalos.columns = ['2.5%', '97.5%']
print(intervalos)

                                                        2.5%     97.5%
Intercept                                           1.091928  1.113953
genero[T.M]                                         0.249099  0.254930
faixa_ano[T.2016-2020]                              0.871460  0.884381
faixa_ano[T.2021-2025]                              1.748623  1.761422
estado_civil[T.outro]                               0.311951  0.343629
estado_civil[T.separado.divorciado]                -0.107762 -0.093641
estado_civil[T.solteiro]                            0.040780  0.055131
profissao[T.Agropecuaria e Meio Ambiente]           0.015522  0.050800
profissao[T.Comunicacao, Artes e Midia]            -0.190802 -0.172731
profissao[T.Direito, Seguranca e Servicos Publi... -0.635620 -0.625095
profissao[T.Educacao, Pesquisa e Ciencias]         -0.457593 -0.447299
profissao[T.Industria, Construcao e Transporte]     0.095164  0.109129
profissao[T.Outros]                                -0.256756 -0.249505
profis

In [23]:
# Odds ratios
odds_ratios = np.exp(modelo.params)
print(odds_ratios)

Intercept                                              3.013013
genero[T.M]                                            1.286615
faixa_ano[T.2016-2020]                                 2.405892
faixa_ano[T.2021-2025]                                 5.783577
estado_civil[T.outro]                                  1.387897
estado_civil[T.separado.divorciado]                    0.904203
estado_civil[T.solteiro]                               1.049124
profissao[T.Agropecuaria e Meio Ambiente]              1.033717
profissao[T.Comunicacao, Artes e Midia]                0.833796
profissao[T.Direito, Seguranca e Servicos Publicos]    0.532402
profissao[T.Educacao, Pesquisa e Ciencias]             0.636070
profissao[T.Industria, Construcao e Transporte]        1.107546
profissao[T.Outros]                                    0.776366
profissao[T.Saude]                                     0.558994
profissao[T.Servicos Gerais e Atendimento]             1.139511
profissao[T.Tecnologia, Engenharia e Pro

In [31]:
# Coeficientes e p-valores do modelo
coef = modelo.params
pvals = modelo.pvalues

# Odds ratios arredondadas com 2 casas decimais
odds_ratios = np.exp(coef).round(2)

# Intervalos de confiança para OR com 4 casas decimais
conf_int_or = np.exp(modelo.conf_int())
conf_int_or.columns = ['IC 2.5%', 'IC 97.5%']
conf_int_or = conf_int_or.round(4)

# Criando o DataFrame final
tabela = pd.DataFrame({
    'Coeficiente': modelo.params.round(4),
    'Erro Padrão': modelo.bse.round(4),
    'p-valor': modelo.pvalues.round(4),
    'IC 2.5%': np.exp(modelo.conf_int())[0].round(4),
    'IC 97.5%': np.exp(modelo.conf_int())[1].round(4),
    'Odds Ratio': np.exp(modelo.params).round(2)
})

print(tabela)

# Exibindo o DataFrame
print(tabela)

                                                     Coeficiente  Erro Padrão  \
Intercept                                                 1.1029       0.0056   
genero[T.M]                                               0.2520       0.0015   
faixa_ano[T.2016-2020]                                    0.8779       0.0033   
faixa_ano[T.2021-2025]                                    1.7550       0.0033   
estado_civil[T.outro]                                     0.3278       0.0081   
estado_civil[T.separado.divorciado]                      -0.1007       0.0036   
estado_civil[T.solteiro]                                  0.0480       0.0037   
profissao[T.Agropecuaria e Meio Ambiente]                 0.0332       0.0090   
profissao[T.Comunicacao, Artes e Midia]                  -0.1818       0.0046   
profissao[T.Direito, Seguranca e Servicos Publicos]      -0.6304       0.0027   
profissao[T.Educacao, Pesquisa e Ciencias]               -0.4524       0.0026   
profissao[T.Industria, Const

In [30]:
# Erro padrão dos coeficientes
std_err = modelo.bse
print(std_err)

Intercept                                              0.005619
genero[T.M]                                            0.001488
faixa_ano[T.2016-2020]                                 0.003296
faixa_ano[T.2021-2025]                                 0.003265
estado_civil[T.outro]                                  0.008081
estado_civil[T.separado.divorciado]                    0.003602
estado_civil[T.solteiro]                               0.003661
profissao[T.Agropecuaria e Meio Ambiente]              0.009000
profissao[T.Comunicacao, Artes e Midia]                0.004610
profissao[T.Direito, Seguranca e Servicos Publicos]    0.002685
profissao[T.Educacao, Pesquisa e Ciencias]             0.002626
profissao[T.Industria, Construcao e Transporte]        0.003563
profissao[T.Outros]                                    0.001850
profissao[T.Saude]                                     0.003200
profissao[T.Servicos Gerais e Atendimento]             0.003124
profissao[T.Tecnologia, Engenharia e Pro